<a href="https://colab.research.google.com/github/crystalloide/Notebooks-SQL-serie-1/blob/main/ex10_Utilisation_des_vues_pour_simplifier_les_requ%C3%AAtes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ex10 - Utilisation des vues pour simplifier les requêtes

L'un des beaux aspects du modèle de données relationnel et SQL est que la sortie d'une requête est également une table, une relation pour être précis. 

Il peut s'agir d'une seule colonne ou d'une seule ligne, mais il s'agit néanmoins d'un tableau. 

Une vue est une requête qui peut être utilisée comme une table. Une vue peut être considérée comme une table virtuelle qui ne contient pas de données. 

Elle correspond juste à une requête. Chaque fois qu'une vue est accédée, la requête sous-jacente est exécutée et les résultats renvoyés peuvent être utilisés comme s'ils constituaient une table réelle.

Il y a plusieurs raisons (http://www.sqlitetutorial.net/sqlite-create-view/) pour utiliser les vues. Gardez à l’esprit le principe de programmation DRY: ne vous répétez pas. Éviter les répétitions permet de gagner du temps et d'éviter les erreurs inutiles. C'est l'une des bonnes raisons pour lesquelles nous enregistrons les requêtes sous forme de vues de base de données réutilisables.

Les vues SQLite sont créées à l'aide de l'instruction CREATE VIEW. 

Les vues SQLite peuvent être créées à partir d'une seule table, de plusieurs tables ou d'une autre vue. 

Voici la syntaxe de la commande ***CREATE VIEW*** de base (http://www.sqlitetutorial.net/sqlite-create-view/):

>CREATE [TEMP | TEMPORARY] VIEW view_name AS<br>
>SELECT column1, column2.....<br>
>FROM table_name<br>
>WHERE [condition];


La vue dans SQLite est en lecture seule. Cela signifie que vous ne pouvez pas utiliser les instructions INSERT, DELETE et UPDATE pour mettre à jour les données dans les tables de base via la vue.


In [1]:
%load_ext sql

In [2]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connection à la database demo.db3

In [3]:
%sql sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3

'Connected: @/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3'

Si vous ne vous souvenez pas des tables présentes dans la database de démonstration, vous pouvez toujours utiliser la commande suivante pour les retrouver.

In [4]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. Simplifier les requêtes avec l'utilisation de vues

Dans le Notebook précédent, nous avons utilisé CASE et Subquery pour calculer le ruissellement saisonnier à partir du tableau de rch. 

Ici, nous utiliserons une vue pour simplifier le calcul.

#### 2.1 Rappel sur le calcul du ruissellement saisonnier

In [5]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM(
SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch)
GROUP BY RCH, Quarter
LIMIT 5

Done.


RCH,Quarter,Runoff
1,DJF,99.20499053531223
1,JJA,1405.262297990587
1,MAM,559.7469320191277
1,SON,454.73798531426326
2,DJF,56.328539085388186


#### 2.2 Création d'une view

In [6]:
%%sql sqlite://
CREATE VIEW RCH_VW AS SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch

Done.


[]

Requêtons maintenant la vue SSN_RCH

In [7]:
%%sql sqlite://
SELECT * 
FROM RCH_VW 
LIMIT 5

Done.


RCH,YR,Quarter,FLOW_OUTcms
1,1981,DJF,146.2524871826172
2,1981,DJF,96.18285369873047
3,1981,DJF,11.861368179321289
4,1981,DJF,49.40651321411133
5,1981,DJF,272.10601806640625


### 2.3 Recalculons le ruissellement avec la vue :

Le code est réellement simplifié et plus court

In [8]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM RCH_VW
GROUP BY RCH, Quarter
LIMIT 5

Done.


RCH,Quarter,Runoff
1,DJF,99.20499053531223
1,JJA,1405.262297990587
1,MAM,559.7469320191277
1,SON,454.73798531426326
2,DJF,56.328539085388186


### 2.4 Suppression de vues

Il est de plus assez facile de supprimer des vues : 


In [9]:
%sql DROP VIEW RCH_VW

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


[]

### Conclusion 

Les vues sont des tables virtuelles qui ne contiennent pas de données, uniquement des instructions SQL. 

Ces instructions sont exécutées à chaque accès à la vue. 

Étant donné que les vues sont créées de manière dynamique lors de leur accès et que les données de ces vues sont toujours fraîches et à jour, elles présentent certains avantages par rapport à la création de sous-tables à partir d'une table. 

Les données des sous-tableaux sont statiques et peuvent être obsolètes.

Une vue est utile dans certains cas (http://www.sqlitetutorial.net/sqlite-create-view/):
- Premièrement, les vues fournissent une couche d'abstraction sur les tables. Vous pouvez ajouter et supprimer des colonnes dans la vue sans toucher au schéma des tables sous-jacentes.
- Deuxièmement, vous pouvez utiliser des vues pour encapsuler des requêtes complexes avec des jointures afin de simplifier l'accès aux données.